In [1]:
__author__ = 'satish'
from bs4 import BeautifulSoup
import re
import urllib2
import pandas as pd
import numpy as np

In [2]:
score_re_pattern = "http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/*"
assist_re_pattern= "http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/*"


In [3]:


def get_page(url):
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page)
    return soup



def get_cols(soup):
    table_div = soup.find(id='my-players-table')
    table = table_div.find("table")
    header_row = table.find(attrs={"class": "colhead"})
    header_cols = header_row.findAll('td')
    cols = []
    for header_col in header_cols:
        val = header_col.string
        if val != 'RK':
            cols.append(val)
        if val == 'PLAYER':
            cols.append('POSITION')
    return cols


def get_data(soup, cols):
    table_div = soup.find(id='my-players-table')
    table = table_div.find("table")
    players_stats_dicts = []
    players_stats_array = []
    rows = table.findAll('tr')
    for row in rows:
        if row.attrs['class'][0] == 'colhead':
            continue
        player_stats = []
        row_tds = row.findAll('td')
        player_col = row_tds[1]
        player_name = player_col.find('a').string
        player_position = player_col.contents[1]
        player_position = player_position.split(' ')[1]
        player_stats.append(player_name)
        player_stats.append(player_position)
        for i in range(2, len(row_tds)):
            stat = row_tds[i].string
            player_stats.append(stat)
        players_stats_array.append(player_stats)
        player_stats = dict(zip(cols, player_stats))
        players_stats_dicts.append(player_stats)
    return players_stats_dicts


def has_next_page(soup):
    page_nums =  soup.find(attrs={"class": "page-numbers"})
    page_num_str = page_nums.string
    pages=page_num_str.split('of')
    start = int(pages[0].strip())
    end = int(pages[1].strip())
    print start,end
    if start==end:
        return False
    return True


def get_next_page_url(soup,traversed_links,pat):
    a_tag = soup.find_all(attrs={"rel": "nofollow"},href = re.compile(pat));
    for tag in a_tag:
        link = tag.get('href')
        if(link in traversed_links):
            continue
        else:
            return link



def scrape_page(url,pattern):
    #get soup object
    bSoup = get_page(url)
    cols = get_cols(bSoup)
    players_stats_dicts =[]
    traversed_links = set()
    hasMorePages= True
    while(has_next_page(bSoup)):
        pl_stats = get_data(bSoup,cols)
        players_stats_dicts.extend(pl_stats)
        traversed_links.add(url)
        url = get_next_page_url(bSoup,traversed_links,pattern)
        print 'next url :: {}'.format(url)
        bSoup = get_page(url)

    return players_stats_dicts



In [4]:
scoring_dicts= scrape_page("http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false",score_re_pattern)
score_df = pd.DataFrame.from_dict(scoring_dicts)

1 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/41
2 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/81
3 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/121
4 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/161
5 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/201
6 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/241
7 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/281
8 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/321
9 13
next url :: h

In [5]:
assists_dicts= scrape_page("http://espn.go.com/nba/statistics/player/_/stat/assists/qualified/false",assist_re_pattern)
assists_df = pd.DataFrame.from_dict(assists_dicts)

1 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/41
2 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/81
3 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/121
4 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/161
5 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/201
6 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/241
7 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/281
8 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/321
9 13
next url :: http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgA

In [13]:
score_df.head()
score_df.columns

Index([        3P%,     3PM-3PA,         FG%,     FGM-FGA,         FT%,
           FTM-FTA,          GP,         MPG,      PLAYER, u'POSITION',
               PTS,        TEAM],
      dtype='object')

In [7]:
assists_df.head()

,AP48M,APG,AST,AST/TO,GP,MPG,PLAYER,POSITION,TEAM,TO,TOPG
0,14.1,10.2,838,4.41,82,34.8,Chris Paul,PG,LAC,190,2.3
1,13.4,10.0,792,2.61,79,35.9,John Wall,PG,WSH,304,3.8
2,13.0,9.6,720,3.89,75,35.5,Ty Lawson,PG,DEN,185,2.5
3,13.4,8.8,193,3.02,22,31.5,Ricky Rubio,PG,MIN,64,2.9
4,12.0,8.6,574,1.96,67,34.4,Russell Westbrook,PG,OKC,293,4.4


In [11]:
assists_df.shape

(480, 11)

In [16]:
merge_df = pd.merge(score_df,assists_df,on = ["PLAYER","TEAM"])
merge_df.columns

Index([          3P%,       3PM-3PA,           FG%,       FGM-FGA,
                 FT%,       FTM-FTA,       u'GP_x',      u'MPG_x',
              PLAYER, u'POSITION_x',           PTS,          TEAM,
               AP48M,           APG,           AST,        AST/TO,
             u'GP_y',      u'MPG_y', u'POSITION_y',            TO,
                TOPG],
      dtype='object')

In [17]:
merge_df[["PLAYER","TEAM","3P%","3PM-3PA","FG%","FGM-FGA","FT%","FTM-FTA","GP_x","GP_y","MPG_y","MPG_x","AP48M","APG","AST","AST/TO","TO","TOPG"]]

,PLAYER,TEAM,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP_x,GP_y,MPG_y,MPG_x,AP48M,APG,AST,AST/TO,TO,TOPG
0,Russell Westbrook,OKC,.299,1.3-4.3,.426,9.4-22.0,.835,8.1-9.8,67,67,34.4,34.4,12.0,8.6,574,1.96,293,4.4
1,James Harden,HOU,.375,2.6-6.9,.440,8.0-18.1,.868,8.8-10.2,81,81,36.8,36.8,9.1,7.0,565,1.76,321,4.0
2,Kevin Durant,OKC,.403,2.4-5.9,.510,8.8-17.3,.854,5.4-6.3,27,27,33.8,33.8,5.8,4.1,110,1.49,74,2.7
3,LeBron James,CLE,.354,1.7-4.9,.488,9.0-18.5,.710,5.4-7.7,69,69,36.1,36.1,9.8,7.4,511,1.88,272,3.9
4,Anthony Davis,NO,.083,0.0-0.2,.535,9.4-17.6,.805,5.5-6.8,68,68,36.1,36.1,2.9,2.2,149,1.57,95,1.4
5,Carmelo Anthony,NY,.341,1.5-4.5,.444,9.0-20.2,.797,4.7-5.9,40,40,35.7,35.7,4.1,3.1,122,1.37,89,2.2
6,DeMarcus Cousins,SAC,.250,0.0-0.1,.467,8.4-18.1,.782,7.2-9.2,59,59,34.1,34.1,5.0,3.6,210,0.83,254,4.3
7,Stephen Curry,GS,.443,3.6-8.1,.487,8.2-16.8,.914,3.9-4.2,80,80,32.7,32.7,11.4,7.7,619,2.49,249,3.1
8,LaMarcus Aldridge,POR,.352,0.5-1.5,.466,9.3-19.9,.845,4.3-5.1,71,71,35.4,35.4,2.4,1.7,124,1.02,122,1.7
9,Kobe Bryant,LAL,.293,1.5-5.3,.373,7.6-20.4,.813,5.6-6.9,35,35,34.5,34.5,7.8,5.6,197,1.54,128,3.7
